In [36]:
from manim import *
import random

config.tex_compiler = "xelatex"  # or "lualatex"
config.frame_width = 4.5
config.frame_height = 8
config.pixel_width = 1080
config.pixel_height = 1920

n_col_x = 10
n_row_y = 10

dot_different_color_index = 47 # index of person wearing different color

# -----------------------------------------------
def DrawScreenFrame(self):

    original_frame_width = 14.2  # Default Manim landscape width
    scale_factor = original_frame_width / config.frame_width  # ~3.15

    frame_border = Rectangle(
    width=config.frame_width,  # Width of the frame
    height=config.frame_height,  # Height of the frame
    color=WHITE,  # Border color
    stroke_width=2  # Border thickness
    )

    # Add the border to the scene
    self.add(frame_border)
    self.play(Create(frame_border)) 

# ---------------------------------------------

def LatexTextWrapping_withLines(self, lines_list, custom_template, start_point = ORIGIN + 2.1*LEFT + 0.7*UP, text_alignment=ORIGIN, text_font_size=15):

    # Create Tex objects for each line
    tex_group = VGroup(*[
        Tex(line, font_size=text_font_size, color=WHITE, tex_template=custom_template) 
        for line in lines_list
    ])

    # Arrange lines vertically with left alignment
    tex_group.arrange(DOWN, aligned_edge=text_alignment, buff=0.1)

    # Position the entire group at a starting point (e.g., LEFT side)
    tex_group.move_to(start_point, aligned_edge=text_alignment)

    self.play(LaggedStart(*[Write(tex) for tex in tex_group], lag_ratio=0.3))
    
# ===================================================================

def DrawGrid(self, dot_grid):

    DrawScreenFrame(self)
    
    grid_box_height = 0.3 * config.frame_height
    
    for i in range(n_col_x):
        for j in range(n_row_y):
            pos_x = -config.frame_width/2 + (i+1)*config.frame_width/(n_col_x+1)
            pos_y = config.frame_height/2 - (j+1)*grid_box_height/(n_row_y+1)
            pos_z = 0
            position = [pos_x, pos_y, pos_z]
            #print(position)
            dot = Dot(point=position, color=YELLOW, radius=0.1)
            dot_grid.add(dot)

    x_grid_border = SurroundingRectangle(
        dot_grid,  # The object to surround
        color=WHITE,  # Border color
        stroke_width=1,  # Border thickness
        fill_color=BLUE,  # Fill color
        fill_opacity=0.5,
        buff=0.05  # Padding around the text
    )

    self.add(dot_grid)
    
    #dot_different_color_index = np.random.randint(0,n_col_x*n_row_y)
    dot_grid[dot_different_color_index].set_color(BLUE)
    #self.play(dot_grid[dot_different_color_index].animate.set_color(RED))

    self.play(Create(dot_grid))
    self.wait(1)
    
    self.add(x_grid_border)
    self.bring_to_front(dot_grid)
    self.play(Create(x_grid_border))
    self.wait(1)


def CountDownTimer(self, timer_time_length = 10, circle_position = ORIGIN+1.5*DOWN):
    # Countdown radius
    countdown_radius = 0.5

    # Countdown textsize
    countdown_text_size = 36
    
    countdown_static_circle = Circle(radius=countdown_radius, stroke_width=1, color=WHITE).move_to(circle_position)
    
    # ValueTracker for time remaining
    time_remaining = ValueTracker(10)
    
    # Dynamic arc (thick stroke) - CLOCKWISE reduction
    countdown_arc = always_redraw(lambda: Arc(
        radius=countdown_radius,
        start_angle=PI/2,
        angle = TAU * (time_remaining.get_value() / timer_time_length),  # TAU = 2π
        stroke_width=5,
        color=BLUE,
        arc_center=countdown_static_circle.get_center()
    ))
    
    # Countdown text (properly sized)
    countdown_text = always_redraw(lambda: Integer(
        int(time_remaining.get_value()+1),
        font_size=countdown_text_size,
        color=WHITE
    ).move_to(countdown_static_circle.get_center()))
    
    # Add all elements to scene
    self.add(countdown_static_circle, countdown_arc, countdown_text)
    
    # Animate over 10 seconds
    self.play(
        time_remaining.animate.set_value(0),
        run_time=10,
        rate_func=linear
    )
    self.wait(0.5)

    self.remove(countdown_static_circle, countdown_arc, countdown_text)

    
# =============================================================

class HundredPersons(Scene):
    def construct(self):

        custom_template = TexTemplate()
        custom_template.add_to_preamble(r"\usepackage{xcolor}")
        #custom_template.add_to_preamble(r"\usepackage{fontspec}")  # For XeLaTeX/LuaLaTeX
        #custom_template.add_to_preamble(r"\setmainfont{Arial}")  # Set custom font

        DrawScreenFrame(self)
        dot_grid = VGroup()
        DrawGrid(self, dot_grid)
                
        # List of sentences to display
        question_text_lines = [
            r"In a room of \colorbox{purple}{\textbf{100}} people,",
            r"\colorbox{purple}{\textbf{99\%}} of them are wearing \textbf{\textcolor{yellow}{YELLOW}} clothes.",
            r"How many people wearing \textbf{\textcolor{yellow}{YELLOW}} must leave",
            r"so that the percentage of people wearing \textbf{\textcolor{yellow}{YELLOW}}",
            r"becomes \colorbox{purple}{\textbf{98\%}}?"
            ]

        LatexTextWrapping_withLines(self, question_text_lines, custom_template, text_alignment=LEFT, text_font_size=15)
        self.wait(3)

        CountDownTimer(self)
        self.wait(2)

#---------------------------------------------------
        answer_font_size = 20
        answer_equations = [
            MathTex(r"\frac{99-x}{100-x} = \frac{98}{100}", font_size=answer_font_size),
            MathTex(r"100(99-x)  =  98(100-x)", font_size=answer_font_size),
            MathTex(r"9900 - 100x  =  9800 - 98x", font_size=answer_font_size),
            MathTex(r"100x - 98x =  9900 - 9800", font_size=answer_font_size),
            MathTex(r"2x =  100", font_size=answer_font_size),
            MathTex(r"x =  50", font_size=answer_font_size)
        ]

        # Position equations with custom spacing
        answer_equations[0].move_to(ORIGIN + 0.5*DOWN)
        for i in range(1, len(answer_equations)):
            answer_equations[i].next_to(answer_equations[i-1], DOWN, aligned_edge=ORIGIN, buff=0.2)

        # Animate sequentially
        for eq in answer_equations:
            self.play(Write(eq))
            self.wait(1)


        answer_border = SurroundingRectangle(
            answer_equations[-1],  # The object to surround
            color=YELLOW,  # Border color
            stroke_width=1.5,  # Border thickness
            #fill_color=GREEN,  # Fill color
            #fill_opacity=0.3,
            buff=0.07  # Padding around the text
        )
        self.add(answer_border)
        self.bring_to_front(answer_equations[-1])
        self.play(Create(answer_border))

        grp2 = VGroup(answer_equations[-1], answer_border)
        self.play(grp2.animate.scale(2))  # Scale up by a factor of 2
        self.wait(2)

# -------------------- Leave
        all_index = range(0, n_col_x*n_row_y)
        all_index_yellow = np.delete(all_index, dot_different_color_index)
        leave_indicies = random.sample(all_index_yellow.tolist(), 50)

        direction_to_leave = [UP, RIGHT, LEFT]

        animations = [
            dot_grid[i].animate.shift(random.choice(direction_to_leave) * 4).fade(1)  # Move up and fade
            for i in leave_indicies]

        self.play(*animations, lag_ratio = 1, run_time=3)
        self.wait(10)
        

In [38]:
manim -pql -r 1080,1920  HundredPersons

Manim Community v0.19.0

[02/10/25 10:43:15] INFO     Animation 0 : Using cached data (hash :                           ]8;id=433287;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=435448;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             4081915702_2357889326_1223600990)                                                     

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=720369;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=619314;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3397231721_1004712769_1575301644)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=657213;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=317170;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3397231721_3797129624_3422442179)                                                     

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=248139;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=135493;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3397231721_4071516364_4101445323)                                                     

...

[02/10/25 10:43:43] INFO                                                                   ]8;id=143763;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=496260;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/mhd/myproj/manim_video/media/videos/manim_video/1920p1                         
                             5/HundredPersons.mp4'                                                                 
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=206333;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=853011;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#856\856]8;;\
                             manim has removed the 3 oldest file(s). You can change this                           
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered HundredPersons                                                   ]8;id=752127;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=821718;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 28 animations                                                                  

                    INFO     Previewed File at:                                                     ]8;id=48927;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/utils/file_ops.py\file_ops.py]8;;\:]8;id=787692;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/utils/file_ops.py#237\237]8;;\
                             '/home/mhd/myproj/manim_video/media/videos/manim_video/1920p15/Hundred                
                             Persons.mp4'                                                                          

[000058e28680ea00] main libvlc: Running vlc with the default interface. Use 'cvlc' to use vlc without interface.
TagLib: MP4: No audio tracks
TagLib: MP4: No audio tracks
Failed to open VDPAU backend libvdpau_nvidia.so: cannot open shared object file: No such file or directory


In [32]:
from manim import *
import random

class RandomDirectionExample(Scene):
    def construct(self):
        # Create an object (e.g., a circle)
        circle = Circle(color=BLUE)
        self.add(circle)

        # List of possible directions
        directions = [UP, RIGHT, LEFT]
        chosen_direction = random.choice(directions)

        # Animate the object moving in the random direction
        self.play(
            circle.animate.shift(chosen_direction * 2),  # Move 2 units in the chosen direction
            run_time=2
        )

        self.wait()

In [33]:
manim -pql RandomDirectionExample

Manim Community v0.19.0

[02/10/25 10:39:01] INFO     Animation 0 : Partial movie file written in                   ]8;id=36253;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=471543;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/mhd/myproj/manim_video/media/videos/manim_video/480p15                         
                             /partial_movie_files/RandomDirectionExample/865358654_2603620                         
                             555_3481380946.mp4'                                                                   

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=118706;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=209353;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/mhd/myproj/manim_video/media/videos/manim_video/480p15                         
                             /partial_movie_files/RandomDirectionExample/3869913882_667144                         
                             672_1051212286.mp4'                                                                   

                    INFO     Combining to Movie file.                                      ]8;id=919147;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=287263;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=812192;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=86831;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/mhd/myproj/manim_video/media/videos/manim_video/480p15                         
                             /RandomDirectionExample.mp4'                                                          
                                                                                                                   

                    INFO     Rendered RandomDirectionExample                                           ]8;id=828100;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=175617;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

                    INFO     Previewed File at:                                                     ]8;id=828483;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/utils/file_ops.py\file_ops.py]8;;\:]8;id=406421;file:///home/mhd/miniconda3/envs/manim_env/lib/python3.9/site-packages/manim/utils/file_ops.py#237\237]8;;\
                             '/home/mhd/myproj/manim_video/media/videos/manim_video/480p15/RandomDi                
                             rectionExample.mp4'                                                                   

[0000644870614a00] main libvlc: Running vlc with the default interface. Use 'cvlc' to use vlc without interface.
TagLib: MP4: No audio tracks
TagLib: MP4: No audio tracks
Failed to open VDPAU backend libvdpau_nvidia.so: cannot open shared object file: No such file or directory
